> # G-17 (best guess single song)

The code is the same written in "G-17_FinalVersion", we just work on the test_golden_song (named "song" in the notebook) set intead of the test set. \
The unique important edit is when we evaluate the mean of the mel frequencies among the 5 audio because we have 5 rows related to the same song.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.svm import SVR

from sklearn.decomposition import KernelPCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


from scipy.signal import find_peaks
from scipy import stats

In [2]:
np.random.seed(0)

In [3]:
train = pd.read_csv('../input/statistical-learning-sapienza-spring-2020/train/train.csv')
song = pd.read_csv('../input/test-golden-song/test_golden_song.csv')


In [ ]:
target = train['tempo']
del train['id']
del train['tempo']
song_id = song['id']
del song['id']

In [7]:
# transform the genre an object type variable
train['genre'] = train['genre'].astype('object')
song['genre'] = song['genre'].astype('object')


In [8]:
# take the Mel frequencies
train_mel = train.iloc[:,87552:94392]
song_mel = song.iloc[:,87552:94392]
song_mel.head()

,mel1_t1,mel1_t2,mel1_t3,mel1_t4,mel1_t5,mel1_t6,mel1_t7,mel1_t8,mel1_t9,mel1_t10,...,mel40_t162,mel40_t163,mel40_t164,mel40_t165,mel40_t166,mel40_t167,mel40_t168,mel40_t169,mel40_t170,mel40_t171
0,290.390085,288.722822,288.042364,291.096707,293.555758,294.327833,293.001358,292.372787,293.699268,292.456072,...,-0.005104,0.885356,-0.294912,0.065527,0.217196,1.083831,0.154642,-0.128929,1.162814,0.845345
1,269.622706,270.125309,270.736975,270.239354,268.313192,267.924079,267.500136,265.882130,264.967248,266.238585,...,-0.212971,-0.045549,-0.602041,0.175532,-0.402959,0.456084,0.309465,0.584882,-0.170607,0.074577
2,264.556740,261.847027,264.186670,266.694661,264.680171,273.278056,275.068353,286.687219,288.950500,292.900308,...,3.453333,2.987845,-0.356687,0.550757,-0.135315,0.387117,-0.252212,-0.313473,-0.543209,0.339174
3,278.713722,282.041832,285.958713,267.971572,284.349512,286.565490,283.451078,277.679791,269.097672,286.768287,...,-0.982109,-0.774137,-1.245132,-0.645180,1.265170,1.589034,0.507678,-0.070205,-0.492837,0.427526
4,249.015522,246.178741,244.544047,245.313894,248.335631,247.954608,249.780833,247.391128,248.044933,243.306980,...,0.568611,1.028450,1.676547,1.805884,0.871031,1.575619,1.434036,1.352388,1.078937,2.005797


In [9]:
# The binning function, that performs the above described operation, takes in input a dataframe d and a binwidth x 
# and gets in output the reduced matrix. 
# It is possible to see that different "group-by" functions has been tried, but we saw that keeping only the mean gave us the best
# results.

def binning(d,x):
    start = 0
    end = x
    diz = {}
    dim = d.shape[1]
    while start < dim:
        if end >= dim:
            temp = d[d.columns[start:dim]]
            mean = temp.mean(axis = 1)
            #median = temp.median(axis=1)
            #minimo = temp.min(axis = 1)
            #massimo = temp.max(axis = 1)
            #std = temp.std(axis=1)
            #diff = massimo-minimo
            diz[str(start)+'_'+str(dim)+'_mean'] = list(mean)
            #diz[str(start)+'_'+str(dim)+'_std'] = list(std)
            #diz[str(start)+'_'+str(dim)+'_median'] = list(median)
            #diz[str(start)+'_'+str(dim)+'_diff'] = list(diff)
            #diz[str(start)+'_'+str(dim)+'_minimo'] = list(minimo)
            #diz[str(start)+'_'+str(dim)+'_massimo'] = list(massimo)
            break
        else:     
            temp = d[d.columns[start:end]]
            mean = temp.mean(axis = 1)
            #median = temp.median(axis=1)
            #minimo = temp.min(axis = 1)
            #massimo = temp.max(axis = 1)
            #std = temp.std(axis=1)
            #diff = massimo-minimo
            diz[str(start)+'_'+str(dim)+'_mean'] = list(mean)
            #diz[str(start)+'_'+str(dim)+'_std'] = list(std)
            #diz[str(start)+'_'+str(dim)+'_median'] = list(median)
            #diz[str(start)+'_'+str(dim)+'_diff'] = list(diff)
            #diz[str(start)+'_'+str(dim)+'_minimo'] = list(minimo)
            #diz[str(start)+'_'+str(dim)+'_massimo'] = list(massimo)
            start += x
            end += x
    final = pd.DataFrame(diz)
    return final

In [10]:
# applying the binning function on train and test and standardizing them

train_mel = preprocessing.scale(binning(train_mel,171))
song_mel = preprocessing.scale(binning(song_mel,171))
song_mel.shape

(5, 40)

First of all we assume that the tempo in the entire song is fixed. We have 5 different pieces of the song and these may be 5 *very* different pieces, in which some of them in the model may not work "well". From this, instead of having 5 different 171x40 matrices we have one matrix of shape 855x40.Following the binning approach we should take the time average for each frequency in the 855x40 matrix reducing the matrix to a vector of shape 1x40. To perform this operation we treated the 5 rows as separated songs until the binning(binning included) and then we took the mean vector between these that in the end represent our final test-golden-song vector.

In [11]:
song_mel =  song_mel.mean(axis = 0)

In [12]:
song_mel = song_mel.reshape(1, -1)
song_mel

array([[ 1.11022302e-15,  3.55271368e-16,  0.00000000e+00,
        -3.10862447e-16,  4.44089210e-17,  6.66133815e-17,
        -3.33066907e-17,  4.44089210e-17,  2.22044605e-16,
        -8.88178420e-17,  1.77635684e-16, -4.44089210e-16,
        -4.44089210e-17, -1.33226763e-16,  2.22044605e-16,
        -4.44089210e-17, -1.33226763e-16, -8.88178420e-17,
        -8.88178420e-17,  0.00000000e+00, -8.88178420e-17,
        -8.88178420e-17, -8.88178420e-17,  8.88178420e-17,
        -4.44089210e-17, -2.22044605e-16, -5.55111512e-17,
         1.33226763e-16,  2.22044605e-16, -1.33226763e-16,
        -4.88498131e-16,  7.54951657e-16, -4.44089210e-17,
        -2.22044605e-16,  1.60982339e-16,  3.88578059e-17,
         4.44089210e-17,  3.55271368e-16,  6.66133815e-17,
        -8.88178420e-17]])

In [13]:
# Applying the kernel pca to the train and test. The cosine kernel with 40 components turned out to work pretty well for us

transformer = KernelPCA(n_components=40, kernel='cosine')
train_mel = transformer.fit_transform(train_mel)
song_mel = transformer.transform(song_mel)
song_mel.shape

(1, 40)

In [14]:
train_mel = pd.DataFrame(train_mel)
song_mel = pd.DataFrame(song_mel)

In [15]:
# concatenate the transformed dataset and adding the dummied genre feature

train_mel = pd.concat([train_mel,pd.get_dummies(train['genre'], prefix='genre')], axis=1)

In [16]:
for i in range(1,21):
    if i == 12:
        song_mel['genre_' + str(i)] = 1
    else:
        song_mel['genre_' + str(i)] = 0
song_mel

,0,1,2,3,4,5,6,7,8,9,...,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18,genre_19,genre_20
0,0.103771,-0.094932,-0.112644,-0.091436,-0.184962,0.163099,0.071701,-0.035763,0.001986,0.296881,...,0,1,0,0,0,0,0,0,0,0


In [17]:
# Model tuning of the parameter(s) in the SVR model. The rbf turned out to work best for us. For each parameter the cross-validated 
# it with 5 folds.

'''
score = []
cs = np.arange(415,425,1)
#cs = [0.1,0.0001,0.00001]
for i in cs:
    reg = SVR(kernel='rbf', C=i,epsilon=0.0001)
    scores = cross_val_score(reg, train_mel, target, scoring='neg_root_mean_squared_error')
    #score.append(scores.mean())
    print(i, -scores.mean())
'''

"\nscore = []\ncs = np.arange(415,425,1)\n#cs = [0.1,0.0001,0.00001]\nfor i in cs:\n    reg = SVR(kernel='rbf', C=i,epsilon=0.0001)\n    scores = cross_val_score(reg, train_mel, target, scoring='neg_root_mean_squared_error')\n    #score.append(scores.mean())\n    print(i, -scores.mean())\n"

In [18]:
# our best parameters

reg = SVR(kernel='rbf', C=421, epsilon=0.0001)
reg.fit(train_mel,target)

SVR(C=421, epsilon=0.0001)

In [19]:
submission = pd.DataFrame({"id":0, "target":reg.predict(song_mel)})
submission.to_csv('submission_song_.csv', index=False)
#submission.round(decimals=0).to_csv('submission_mel_round.csv', index=False)
print('The tempo estimate for the golden song is:', reg.predict(song_mel))

The tempo estimate for the golden song is: [123.20087959]
